# Detect Model
## Step 3 - Deploy Endpoint as TensorFlow Model

ref:  https://aws.amazon.com/blogs/machine-learning/deploy-trained-keras-or-tensorflow-models-using-amazon-sagemaker/


In [1]:
import boto3, re
from sagemaker import get_execution_role
from sagemaker.tensorflow.model import TensorFlowModel

from PIL import Image

In [2]:
role = get_execution_role()

In [3]:
S3_SAGEMAKER = "s3://sagemaker-us-east-1-586454201570/"
SAGEMAKER_JOB = "cfa-products-mobilenet-v1-SSD-2019-08-03-20-36-17-231/"
MODEL_OUTPUT = "output/model.tar.gz"

FULL_MODEL_PATH = S3_SAGEMAKER + SAGEMAKER_JOB + MODEL_OUTPUT
! aws s3 ls {FULL_MODEL_PATH}

SAMPLE_IMAGE = "/home/ec2-user/SageMaker/ssd-dag/data/new_jpeg_images/20190710_variety_1562781002.jpg"

2019-08-03 20:59:28  127054873 model.tar.gz


In [8]:
# 20190804
# DON'T EVEN THINK ABOUT THE OBVIOUS!
#  py_version='py3'
#  framework_version='1.13'

sagemaker_model = TensorFlowModel(model_data = FULL_MODEL_PATH,
                    role = role,
                    framework_version='1.12',
                    entry_point = 'code/train.py')

The Python 2 tensorflow images will be soon deprecated and may not be supported for newer upcoming versions of the tensorflow images.
Please set the argument "py_version='py3'" to use the Python 3 tensorflow image.


In [9]:
predictor = sagemaker_model.deploy(initial_instance_count=1,
                                   instance_type='ml.p2.xlarge')

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------*

ValueError: Error hosting endpoint sagemaker-tensorflow-2019-08-04-21-50-09-452: Failed Reason:  The primary container for production variant AllTraffic did not pass the ping health check. Please check CloudWatch logs for this endpoint.

## Grab an Image

In [ ]:
# Helper Function
# load an image and resturn a numpy array

def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [ ]:
image = Image.open(image_path)
image_np = load_image_into_numpy_array(image)
# Expand dimensions since the model expects images to have shape: [1, None, None, 3]
image_np_expanded = np.expand_dims(image_np, axis=0)